In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../dataset/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../dataset/classes-fixed.json
../dataset/classes-fixed.txt
../dataset/classes.txt
../dataset/datasets.desktop
../dataset/GOLD_XYZ_OSC.0001_1024.hdf5
../dataset/LICENSE.TXT


In [1]:
import h5py
import numpy as np
import os,random
from tensorflow.keras.layers import Input,Add,Reshape,AveragePooling2D,ZeroPadding2D,BatchNormalization,MaxPool2D,Conv1D,Conv2D,Dropout,Flatten,Dense,Activation,MaxPooling2D,AlphaDropout
from tensorflow.keras import layers
import tensorflow.keras.models as Model
from tensorflow.keras.regularizers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import GlorotUniform as glorot_uniform
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

In [2]:
f = h5py.File("../dataset/GOLD_XYZ_OSC.0001_1024.hdf5")
temp = -20
count = 0
for i, data in enumerate(f['Z'][:]):
    if temp != data[0]:
        print( f"{i}, {data}")
        temp = data[0]
        count +=1
    if i == 0:
        print( f"{i}, {data}")

print(len(f['X']))  

0, [-20]
4096, [-18]
8192, [-16]
12288, [-14]
16384, [-12]
20480, [-10]
24576, [-8]
28672, [-6]
32768, [-4]
36864, [-2]
40960, [0]
45056, [2]
49152, [4]
53248, [6]
57344, [8]
61440, [10]
65536, [12]
69632, [14]
73728, [16]
77824, [18]
81920, [20]
86016, [22]
90112, [24]
94208, [26]
98304, [28]
102400, [30]
106496, [-20]
110592, [-18]
114688, [-16]
118784, [-14]
122880, [-12]
126976, [-10]
131072, [-8]
135168, [-6]
139264, [-4]
143360, [-2]
147456, [0]
151552, [2]
155648, [4]
159744, [6]
163840, [8]
167936, [10]
172032, [12]
176128, [14]
180224, [16]
184320, [18]
188416, [20]
192512, [22]
196608, [24]
200704, [26]
204800, [28]
208896, [30]
212992, [-20]
217088, [-18]
221184, [-16]
225280, [-14]
229376, [-12]
233472, [-10]
237568, [-8]
241664, [-6]
245760, [-4]
249856, [-2]
253952, [0]
258048, [2]
262144, [4]
266240, [6]
270336, [8]
274432, [10]
278528, [12]
282624, [14]
286720, [16]
290816, [18]
294912, [20]
299008, [22]
303104, [24]
307200, [26]
311296, [28]
315392, [30]
319488, [-20]


# Splitting into half dataset

In [41]:
106496 * 241

25665536

In [31]:
1277952/26

49152.0

In [51]:
2555904/3

851968.0

In [53]:
851968.0/26

32768.0

In [48]:
f = h5py.File("../dataset/GOLD_XYZ_OSC.0001_1024.hdf5")
f['Z'].shape

(2555904, 1)

In [4]:
851968 * 3

2555904

In [3]:
f = h5py.File("../dataset/GOLD_XYZ_OSC.0001_1024.hdf5")

for modu in range(3):
    X_list = []
    Y_list = []
    Z_list = []
    start_modu = modu * 851968 # nilai 106496 merupakan jumlah frame total dari semua SNR untuk 
    # Ada 26 SNR berbeda
    for snr in range(26):
        start_snr = start_modu + snr * 32768
        # print(f"{start_modu}, {start_snr}")
        idx_list = np.random.choice(range(0,32768), size=32768,replace=False)
        
        X_list.append(f['X'][start_snr:start_snr+32768][idx_list])
        Y_list.append(f['Y'][start_snr:start_snr+32768][idx_list])
        Z_list.append(f['Z'][start_snr:start_snr+32768][idx_list])

    filename = "../dataset" + '/half_dataset_part' + str(modu) + '.h5'
    fw = h5py.File(filename,'w')
    fw['X'] = np.vstack(X_list)
    fw['Y'] = np.vstack(Y_list)
    fw['Z'] = np.vstack(Z_list)
    print('X shape:',fw['X'].shape)
    print('Y shape:',fw['Y'].shape)
    print('Z shape:',fw['Z'].shape)
    fw.close()
f.close()

X shape: (851968, 1024, 2)
Y shape: (851968, 24)
Z shape: (851968, 1)
X shape: (851968, 1024, 2)
Y shape: (851968, 24)
Z shape: (851968, 1)
X shape: (851968, 1024, 2)
Y shape: (851968, 24)
Z shape: (851968, 1)


# Splitting into 17 Datasets

In [7]:
f = h5py.File("../dataset/GOLD_XYZ_OSC.0001_1024.hdf5")
modu_snr_size = 1200

for modu in range(2):
    X_list = []
    Y_list = []
    Z_list = []
    start_modu = modu * 1277952.0 # nilai 106496 merupakan jumlah frame total dari semua SNR untuk 
    for snr in range(26):
        start_snr = start_modu + snr*4096
        idx_list = np.random.choice(range(0,4096),size=4096,replace=False)
        X = f['X'][start_snr:start_snr + 4096][idx_list]
		#X = X[:,0:768,:]
        X_list.append(X)
        Y_list.append(f['Y'][start_snr:start_snr + 4096][idx_list])
        Z_list.append(f['Z'][start_snr:start_snr + 4096][idx_list])

    filename = "../dataset" + '/part' + str(modu) + '.h5'
    fw = h5py.File(filename,'w')
    fw['X'] = np.vstack(X_list)
    fw['Y'] = np.vstack(Y_list)
    fw['Z'] = np.vstack(Z_list)
    print('X shape:',fw['X'].shape)
    print('Y shape:',fw['Y'].shape)
    print('Z shape:',fw['Z'].shape)
    fw.close()
f.close()

TypeError: slice indices must be integers or None or have an __index__ method

In [16]:
106496/4096

26.0

In [15]:
f = h5py.File("../dataset/half_dataset_part1.h5")
print(f"x : {f['X']}, Y : {f['Y']}, Z : {[f['Z']]}")

x : <HDF5 dataset "X": shape (106496, 1024, 2), type "<f4">, Y : <HDF5 dataset "Y": shape (106496, 24), type "<i8">, Z : [<HDF5 dataset "Z": shape (106496, 1), type "<i8">]


In [6]:
2555904/2

1277952.0